In [1]:
import pandas as pd

In [2]:
df=pd.read_excel('tst_tr_chegeka.xlsx')

In [3]:
df

,Unnamed: 0,q,a
0,0,В одном из эпизодов мультфильма Криса Батлера ...,"сэр, сир, sir"
1,1,В коротком произведении Елены Кацюбы в аду зло...,удав
2,2,По сюжету фильма Дэвида Хьюитта четверо астрон...,"Дороти, Элли"
3,3,"Дэвид Вуттон пишет, что раньше одни учёные пол...","игра Камень ножницы бумага, Камень ножницы бумага"
4,4,Ведущие украинской телепередачи “Орёл и решка”...,"Дома лучше, А дома лучше"
...,...,...,...
836,847,Животные рода хрисаора также известны как морс...,"Медуса, Медуза"
837,848,"Альберто Анджела пишет, что если проследить пе...",монета
838,849,В казахских танцах движения рук имеют важную р...,"на себя, от себя"
839,850,Начало карьеры Линдси Лохан в кино было по-нас...,"диснеевские принцессы, принцессы Дисней"


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
train, test = train_test_split(df,test_size=400, random_state=42)

In [6]:
base_string='Вы участвуете в викторине “Что? Где? Когда?”. Внимательно прочитайте вопрос и ответьте на него. Обратите внимание на формат ответа: если он не указан в вопросе, то запишите только слово или фразу, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно. Вопрос:'

# Инференс

In [7]:
# pip install --upgrade pip

In [8]:
# pip install wheel #### Нужно для pip install yandexgpt-python

In [9]:
# pip install yandexgpt-python
#from env_vars import *

In [10]:
from yandex_gpt import YandexGPTConfigManagerForAPIKey

# Initialize the configuration manager with model type, catalog ID, and API key
config_manager = YandexGPTConfigManagerForAPIKey(
    model_type="yandexgpt",
    catalog_id=CATALOG_ID,
    api_key=API_KEY
)

In [11]:
from yandex_gpt import YandexGPT

# Initialize YandexGPT with a configuration manager
yandex_gpt = YandexGPT(config_manager=config_manager)

In [12]:
from time import sleep
def query_gpt(row, prompt=base_string):
    completion = yandex_gpt.get_sync_completion(messages=[{'role': 'user', 'text': base_string+row['q'] + ' Ответ:' }])
    sleep(1)
    return completion

In [13]:
test 

,Unnamed: 0,q,a
485,492,Автор детективов Иэн Рэнкин увлекается неким з...,"собирать пазл, собирать мозаику, складывать па..."
817,828,Некоторые виды мух называют путниками из-за их...,"спутниками, спутники, спутник"
30,33,Хави Повес активно продвигает концепцию плоско...,"астронавт, космонавт"
551,560,Борющаяся с музыкальными корпорациями группа C...,троянский конь
96,100,Голландский шахматист Мануэль Босбом выиграл с...,"дырка, дыра"
...,...,...,...
289,294,"Василий Рóзанов писал, что Наталья Нóрдман пог...",пылесос
238,243,Математик семнадцатого века Джон ВАллис активн...,"курица, куриная лапа, лапа курицы, лапка куриц..."
0,0,В одном из эпизодов мультфильма Криса Батлера ...,"сэр, сир, sir"
832,843,В стихотворении под названием «ОН» Дэвид Лоуре...,"комары, комар, комарик"


In [14]:
test['predict'] = test.apply(query_gpt, axis=1)

In [15]:
test.to_csv('test_results.csv')

In [16]:
len(test)

400

In [17]:
test

,Unnamed: 0,q,a,predict
485,492,Автор детективов Иэн Рэнкин увлекается неким з...,"собирать пазл, собирать мозаику, складывать па...",Раскладывать пасьянс.
817,828,Некоторые виды мух называют путниками из-за их...,"спутниками, спутники, спутник",расстоянии
30,33,Хави Повес активно продвигает концепцию плоско...,"астронавт, космонавт",Космонавт
551,560,Борющаяся с музыкальными корпорациями группа C...,троянский конь,Прогрессирующий параноик.
96,100,Голландский шахматист Мануэль Босбом выиграл с...,"дырка, дыра",Надпись.
...,...,...,...,...
289,294,"Василий Рóзанов писал, что Наталья Нóрдман пог...",пылесос,Каспийское море.
238,243,Математик семнадцатого века Джон ВАллис активн...,"курица, куриная лапа, лапа курицы, лапка куриц...",ЕЁ — курица.
0,0,В одном из эпизодов мультфильма Криса Батлера ...,"сэр, сир, sir",«Улыбнись!»
832,843,В стихотворении под названием «ОН» Дэвид Лоуре...,"комары, комар, комарик",Элли


# Вычисление итоговой метрики

In [18]:
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()

    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)

    common_tokens = set(pred_tokens) & set(truth_tokens)

    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0

    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)

    return 2 * (prec * rec) / (prec + rec)

In [19]:
def metrics_calculation(df):
    df['a_list']=df.apply(lambda x: x['a'].split(','), axis=1)
    df['f1_score']=df.apply(lambda x: max([compute_f1(x['predict'], elem) for elem in x['a_list']]), axis=1)
    df['exact_match']=df.apply(lambda x: max([compute_exact_match(x['predict'], elem) for elem in x['a_list']]), axis=1)
    return df

In [20]:
test=metrics_calculation(test)
total_f1_score=test['f1_score'].mean()
total_exact_match=test['exact_match'].mean()

In [38]:
total_f1_score

0.30016666666666664

In [39]:
total_exact_match

0.2575

In [21]:
test

,Unnamed: 0,q,a,predict,a_list,f1_score,exact_match
485,492,Автор детективов Иэн Рэнкин увлекается неким з...,"собирать пазл, собирать мозаику, складывать па...",Раскладывать пасьянс.,"[собирать пазл, собирать мозаику, складывать...",0.0,0
817,828,Некоторые виды мух называют путниками из-за их...,"спутниками, спутники, спутник",расстоянии,"[спутниками, спутники, спутник]",0.0,0
30,33,Хави Повес активно продвигает концепцию плоско...,"астронавт, космонавт",Космонавт,"[астронавт, космонавт]",1.0,1
551,560,Борющаяся с музыкальными корпорациями группа C...,троянский конь,Прогрессирующий параноик.,[троянский конь],0.0,0
96,100,Голландский шахматист Мануэль Босбом выиграл с...,"дырка, дыра",Надпись.,"[дырка, дыра]",0.0,0
...,...,...,...,...,...,...,...
289,294,"Василий Рóзанов писал, что Наталья Нóрдман пог...",пылесос,Каспийское море.,[пылесос],0.0,0
238,243,Математик семнадцатого века Джон ВАллис активн...,"курица, куриная лапа, лапа курицы, лапка куриц...",ЕЁ — курица.,"[курица, куриная лапа, лапа курицы, лапка к...",0.5,0
0,0,В одном из эпизодов мультфильма Криса Батлера ...,"сэр, сир, sir",«Улыбнись!»,"[сэр, сир, sir]",0.0,0
832,843,В стихотворении под названием «ОН» Дэвид Лоуре...,"комары, комар, комарик",Элли,"[комары, комар, комарик]",0.0,0


Самостоятельная оценка качества моделью

In [24]:
base_string2 = "Ты профеесиональный игрок в ЧТо? Где? Когда? Я покажу тебе вопрос из Что? Где? Когда? с  правильным ответом к нему. Еще я подам ответ, который дала команда. Тебе нужно СРАВНИТЬ правильный ответ с ответом команды и ответить, можно ли зачесть ответ команды. Выведи свой ответ в таком формате: Сначала рассуждения, почему ты принял это решение. Потом ответ: #ВЕРНО, если ответ можно зачесть или #НЕВЕРНО, если нельзя. "


In [27]:
from tqdm import tqdm

tqdm.pandas()
def query_gpt_answer(row, prompt=base_string2):
    completion = yandex_gpt.get_sync_completion(messages=[{'role': 'user', 'text': prompt+row['q'] + ' Правильный ответ:'+ row['a'] + 'Ответ команды:' + row['predict'] }])
    sleep(1)
    return completion
test['answer_checking'] =  test.progress_apply(query_gpt_answer, axis=1)

100%|██████████| 400/400 [18:27<00:00,  2.77s/it]


In [28]:
%pip install asyncio
import asyncio

async def apply_async(df, func):
    loop = asyncio.get_event_loop()
    tasks = [loop.create_task(func(row)) for _, row in df.iterrows()]
    return await asyncio.gather(*tasks)


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 2.0 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [29]:
test

,Unnamed: 0,q,a,predict,a_list,f1_score,exact_match,answer_checking
485,492,Автор детективов Иэн Рэнкин увлекается неким з...,"собирать пазл, собирать мозаику, складывать па...",Раскладывать пасьянс.,"[собирать пазл, собирать мозаику, складывать...",0.0,0,"#НЕВЕРНО\n\nОтвет команды нельзя зачесть, поск..."
817,828,Некоторые виды мух называют путниками из-за их...,"спутниками, спутники, спутник",расстоянии,"[спутниками, спутники, спутник]",0.0,0,#НЕВЕРНО.\nРассуждения: правильный ответ — «сп...
30,33,Хави Повес активно продвигает концепцию плоско...,"астронавт, космонавт",Космонавт,"[астронавт, космонавт]",1.0,1,#ВЕРНО\n\nОтвет команды «Космонавт» можно заче...
551,560,Борющаяся с музыкальными корпорациями группа C...,троянский конь,Прогрессирующий параноик.,[троянский конь],0.0,0,#НЕВЕРНО\n\nОтвет команды не подходит к вопрос...
96,100,Голландский шахматист Мануэль Босбом выиграл с...,"дырка, дыра",Надпись.,"[дырка, дыра]",0.0,0,#НЕВЕРНО\n\nКоманда предложила вариант «надпис...
...,...,...,...,...,...,...,...,...
289,294,"Василий Рóзанов писал, что Наталья Нóрдман пог...",пылесос,Каспийское море.,[пылесос],0.0,0,#НЕВЕРНО\nРассуждения: Василий Розанов писал о...
238,243,Математик семнадцатого века Джон ВАллис активн...,"курица, куриная лапа, лапа курицы, лапка куриц...",ЕЁ — курица.,"[курица, куриная лапа, лапа курицы, лапка к...",0.5,0,#ВЕРНО\nРассуждения: ответ команды совпадает с...
0,0,В одном из эпизодов мультфильма Криса Батлера ...,"сэр, сир, sir",«Улыбнись!»,"[сэр, сир, sir]",0.0,0,#НЕВЕРНО\n\nОтвет команды не подходит по смысл...
832,843,В стихотворении под названием «ОН» Дэвид Лоуре...,"комары, комар, комарик",Элли,"[комары, комар, комарик]",0.0,0,#НЕВЕРНО\n\nВ стихотворении Дэвида Лоуренса ре...


In [30]:
test['correct'] = test['answer_checking'].apply(lambda x: '#ВЕРНО' in x)
print(len(test))
test

400


,Unnamed: 0,q,a,predict,a_list,f1_score,exact_match,answer_checking,correct
485,492,Автор детективов Иэн Рэнкин увлекается неким з...,"собирать пазл, собирать мозаику, складывать па...",Раскладывать пасьянс.,"[собирать пазл, собирать мозаику, складывать...",0.0,0,"#НЕВЕРНО\n\nОтвет команды нельзя зачесть, поск...",False
817,828,Некоторые виды мух называют путниками из-за их...,"спутниками, спутники, спутник",расстоянии,"[спутниками, спутники, спутник]",0.0,0,#НЕВЕРНО.\nРассуждения: правильный ответ — «сп...,False
30,33,Хави Повес активно продвигает концепцию плоско...,"астронавт, космонавт",Космонавт,"[астронавт, космонавт]",1.0,1,#ВЕРНО\n\nОтвет команды «Космонавт» можно заче...,True
551,560,Борющаяся с музыкальными корпорациями группа C...,троянский конь,Прогрессирующий параноик.,[троянский конь],0.0,0,#НЕВЕРНО\n\nОтвет команды не подходит к вопрос...,False
96,100,Голландский шахматист Мануэль Босбом выиграл с...,"дырка, дыра",Надпись.,"[дырка, дыра]",0.0,0,#НЕВЕРНО\n\nКоманда предложила вариант «надпис...,False
...,...,...,...,...,...,...,...,...,...
289,294,"Василий Рóзанов писал, что Наталья Нóрдман пог...",пылесос,Каспийское море.,[пылесос],0.0,0,#НЕВЕРНО\nРассуждения: Василий Розанов писал о...,False
238,243,Математик семнадцатого века Джон ВАллис активн...,"курица, куриная лапа, лапа курицы, лапка куриц...",ЕЁ — курица.,"[курица, куриная лапа, лапа курицы, лапка к...",0.5,0,#ВЕРНО\nРассуждения: ответ команды совпадает с...,True
0,0,В одном из эпизодов мультфильма Криса Батлера ...,"сэр, сир, sir",«Улыбнись!»,"[сэр, сир, sir]",0.0,0,#НЕВЕРНО\n\nОтвет команды не подходит по смысл...,False
832,843,В стихотворении под названием «ОН» Дэвид Лоуре...,"комары, комар, комарик",Элли,"[комары, комар, комарик]",0.0,0,#НЕВЕРНО\n\nВ стихотворении Дэвида Лоуренса ре...,False


In [31]:
test.to_csv('test_res_with_bin.csv')

In [32]:
test['correct'].sum()/ len(test['correct'])

0.3375

In [33]:
row = test.iloc[27]
print(row)
completion = yandex_gpt.get_sync_completion(messages=[{'role': 'user', 'text': base_string2+row['q'] + ' ПРАВИЛЬНЫЙ ответ:'+ row['a'] + 'Ответ команды:' + row['predict'] }])
completion

Unnamed: 0                                                       298
q                  В терминологии NASA [нáса] ОНИ называются «оде...
a                                               подгузники, памперсы
predict                                        космические скафандры
a_list                                       [подгузники,  памперсы]
f1_score                                                         0.0
exact_match                                                        0
answer_checking    #НЕВЕРНО\n\nОтвет команды нельзя зачесть, пото...
correct                                                        False
Name: 292, dtype: object


'#НЕВЕРНО\nРассуждения: подгузники и памперсы действительно можно назвать «одеждой с максимальным поглощением», но космические скафандры — это средство защиты, а не одежда. Кроме того, названия популярных моделей подгузников и памперсов переводятся как «баловники» и «обнимашки», что не подходит к космическим скафандрам.'

In [34]:
row['q']

'В терминологии NASA [нáса] ОНИ называются «одеждой с максимальным поглощением». Названия самых популярных моделей ИХ переводятся как «баловники» и «обнимашки». Назовите ИХ.'

In [35]:
df

,Unnamed: 0,q,a
0,0,В одном из эпизодов мультфильма Криса Батлера ...,"сэр, сир, sir"
1,1,В коротком произведении Елены Кацюбы в аду зло...,удав
2,2,По сюжету фильма Дэвида Хьюитта четверо астрон...,"Дороти, Элли"
3,3,"Дэвид Вуттон пишет, что раньше одни учёные пол...","игра Камень ножницы бумага, Камень ножницы бумага"
4,4,Ведущие украинской телепередачи “Орёл и решка”...,"Дома лучше, А дома лучше"
...,...,...,...
836,847,Животные рода хрисаора также известны как морс...,"Медуса, Медуза"
837,848,"Альберто Анджела пишет, что если проследить пе...",монета
838,849,В казахских танцах движения рук имеют важную р...,"на себя, от себя"
839,850,Начало карьеры Линдси Лохан в кино было по-нас...,"диснеевские принцессы, принцессы Дисней"


In [36]:
total_f1_score

0.30016666666666664

In [37]:
total_exact_match

0.2575

In [5]:
data=pd.read_csv('test_res_with_bin.csv')

In [6]:
data['f1_score'].head(40).mean()

0.21845238095238093

In [8]:
data['exact_match'].head(40).mean()

0.175

In [9]:
data['correct'].head(40).sum()/ len(data.head(40)['correct'])

0.325